In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

import tensorflow as tf

In [2]:
turbine_data= pd.read_csv('\\Users\\piyus\\Downloads\\gas_turbines.csv')

In [3]:
# copy the data 
df= turbine_data.copy()

In [4]:
df.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


In [5]:
df.isna().sum()

AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TIT     0
TAT     0
TEY     0
CDP     0
CO      0
NOX     0
dtype: int64

In [6]:
df.columns

Index(['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'TEY', 'CDP', 'CO',
       'NOX'],
      dtype='object')

In [7]:
x= df[['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'CDP', 'CO', 'NOX']]
print(x)
y= df['TEY']
print(y)

           AT      AP      AH    AFDP    GTEP     TIT     TAT     CDP      CO  \
0      6.8594  1007.9  96.799  3.5000  19.663  1059.2  550.00  10.605  3.1547   
1      6.7850  1008.4  97.118  3.4998  19.728  1059.3  550.00  10.598  3.2363   
2      6.8977  1008.8  95.939  3.4824  19.779  1059.4  549.87  10.601  3.2012   
3      7.0569  1009.2  95.249  3.4805  19.792  1059.6  549.99  10.606  3.1923   
4      7.3978  1009.7  95.150  3.4976  19.765  1059.7  549.98  10.612  3.2484   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
15034  9.0301  1005.6  98.460  3.5421  19.164  1049.7  546.21  10.400  4.5186   
15035  7.8879  1005.9  99.093  3.5059  19.414  1046.3  543.22  10.433  4.8470   
15036  7.2647  1006.3  99.496  3.4770  19.530  1037.7  537.32  10.483  7.9632   
15037  7.0060  1006.8  99.008  3.4486  19.377  1043.2  541.24  10.533  6.2494   
15038  6.9279  1007.2  97.533  3.4275  19.306  1049.9  545.85  10.583  4.9816   

          NOX  
0      82.7

In [8]:
# Standerdize the independant variables
x_standerdized= (x-x.mean())/x.std()

In [9]:
x_standerdized

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX
0,-1.439730,-0.826616,1.281394,-0.921201,-1.379055,-1.488326,0.585221,-1.357286,0.531994,1.387799
1,-1.449553,-0.748623,1.304521,-0.921464,-1.363482,-1.482276,0.585221,-1.363631,0.568715,1.392956
2,-1.434674,-0.686227,1.219045,-0.944353,-1.351264,-1.476226,0.568696,-1.360911,0.552919,1.363540
3,-1.413655,-0.623832,1.169021,-0.946853,-1.348149,-1.464125,0.583950,-1.356379,0.548914,1.382832
4,-1.368648,-0.545839,1.161844,-0.924358,-1.354618,-1.458074,0.582678,-1.350940,0.574160,1.348546
...,...,...,...,...,...,...,...,...,...,...
15034,-1.153144,-1.185388,1.401813,-0.865821,-1.498607,-2.063115,0.103449,-1.543109,1.145754,1.085714
15035,-1.303943,-1.138592,1.447705,-0.913440,-1.438712,-2.268829,-0.276629,-1.513196,1.293535,1.119905
15036,-1.386220,-1.076197,1.476921,-0.951457,-1.410920,-2.789165,-1.026616,-1.467873,2.695835,2.169990
15037,-1.420375,-0.998203,1.441542,-0.988815,-1.447576,-2.456392,-0.528319,-1.422551,1.924619,2.391085


# Tunig - Batch size and epochs

In [10]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [11]:
# create a sequential model
def create_model():
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dense(1, kernel_initializer= 'normal')) 
  adam= Adam()
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

In [12]:
# create regression model
model= KerasRegressor(build_fn=create_model, verbose=0)
# create grid search model
batch_size=[50,100, 200, 400, 800, 1000]
epochs=[10,50,100]

param_grid= dict(batch_size= batch_size, epochs=epochs)
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x_standerdized,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5; 1/18] START batch_size=50, epochs=10...................................
[CV 1/5; 1/18] END .batch_size=50, epochs=10;, score=-144.523 total time=   3.1s
[CV 2/5; 1/18] START batch_size=50, epochs=10...................................
[CV 2/5; 1/18] END ..batch_size=50, epochs=10;, score=-19.296 total time=   2.5s
[CV 3/5; 1/18] START batch_size=50, epochs=10...................................
[CV 3/5; 1/18] END ..batch_size=50, epochs=10;, score=-36.113 total time=   2.3s
[CV 4/5; 1/18] START batch_size=50, epochs=10...................................
[CV 4/5; 1/18] END ..batch_size=50, epochs=10;, score=-53.503 total time=   2.3s
[CV 5/5; 1/18] START batch_size=50, epochs=10...................................
[CV 5/5; 1/18] END ..batch_size=50, epochs=10;, score=-63.093 total time=   2.5s
[CV 1/5; 2/18] START batch_size=50, epochs=50...................................
[CV 1/5; 2/18] END ...batch_size=50, epochs=50;,

In [13]:
# results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.7041289865970611, using {'batch_size': 50, 'epochs': 100}
-63.30577125549316,43.28668489954743 with: {'batch_size': 50, 'epochs': 10}
-0.7210009813308715,0.20108191199259495 with: {'batch_size': 50, 'epochs': 50}
-0.7041289865970611,0.2772733811958799 with: {'batch_size': 50, 'epochs': 100}
-225.6418670654297,71.06148987066629 with: {'batch_size': 100, 'epochs': 10}
-2.7037752509117126,1.3694489963688845 with: {'batch_size': 100, 'epochs': 50}
-0.7158853709697723,0.25594338417901097 with: {'batch_size': 100, 'epochs': 100}
-699.9783996582031,331.2900433691676 with: {'batch_size': 200, 'epochs': 10}
-37.12653350830078,19.102650485958016 with: {'batch_size': 200, 'epochs': 50}
-2.595403218269348,1.6694650064337675 with: {'batch_size': 200, 'epochs': 100}
-7020.332568359375,4180.710046947521 with: {'batch_size': 400, 'epochs': 10}
-174.19560699462892,102.57658923435301 with: {'batch_size': 400, 'epochs': 50}
-37.142595291137695,12.538952170195017 with: {'batch_size': 400, 'epochs'

# Tuning : Learning rate and drop out rate

In [14]:
from keras.layers import Dropout

# defining model
def create_model_lr_drate(learning_rate,dropout_rate):
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8, kernel_initializer= 'normal', activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1, kernel_initializer= 'normal')) 
  adam= Adam(learning_rate= learning_rate)
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_lr_drate, verbose=0, batch_size=100, epochs=100)
learning_rate=[0.001, 0.01, 0.1, 0.2, 0.5]
dropout_rate=[0, 0.1, 0.2]

param_grid= dict(learning_rate= learning_rate, dropout_rate=dropout_rate)
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x_standerdized,y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 1/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-0.775 total time=  10.7s
[CV 2/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 2/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-0.549 total time=  11.1s
[CV 3/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 3/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-1.187 total time=  10.5s
[CV 4/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 4/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-0.435 total time=  11.0s
[CV 5/5; 1/15] START dropout_rate=0, learning_rate=0.001........................
[CV 5/5; 1/15] END dropout_rate=0, learning_rate=0.001;, score=-0.850 total time=  10.9s
[CV 1/5; 2/15] START dropout_rate=0, learning_rate=0.01.........................
[CV 1/5;

In [15]:
#  results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.7591996669769288, using {'dropout_rate': 0, 'learning_rate': 0.001}
-0.7591996669769288,0.2608899762068145 with: {'dropout_rate': 0, 'learning_rate': 0.001}
-0.9926034331321716,0.4000795237317309 with: {'dropout_rate': 0, 'learning_rate': 0.01}
-0.8700807213783264,0.2070646105477391 with: {'dropout_rate': 0, 'learning_rate': 0.1}
-3.2180381774902345,3.9798166516043363 with: {'dropout_rate': 0, 'learning_rate': 0.2}
-114.11931080818177,99.5621907573339 with: {'dropout_rate': 0, 'learning_rate': 0.5}
-19.538070678710938,9.520538930791087 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
-15.610881567001343,12.936365410022361 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
-16.72438907623291,7.513736565242277 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
-22.50228958129883,7.644647686421328 with: {'dropout_rate': 0.1, 'learning_rate': 0.2}
-176.81524963378905,106.80224578115381 with: {'dropout_rate': 0.1, 'learning_rate': 0.5}
-43.41292037963867,18.509059971373315 with: {

# Tuning : Activation function and Kernal intitilizer

In [16]:
def create_model_af_ker_ini(activation_function,init):
  model= Sequential()
  model.add(Dense(10, input_dim=10, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= init, activation=activation_function))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= init, activation='linear')) 
  adam= Adam(learning_rate= 0.001)
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_af_ker_ini, verbose=0, batch_size=100, epochs=100)
activation_function=['linear', 'relu']
init=['uniform', 'normal', 'zero']

param_grid= dict(activation_function= activation_function, init=init)
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x_standerdized,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START activation_function=linear, init=uniform....................
[CV 1/5; 1/6] END activation_function=linear, init=uniform;, score=-0.806 total time=  10.8s
[CV 2/5; 1/6] START activation_function=linear, init=uniform....................
[CV 2/5; 1/6] END activation_function=linear, init=uniform;, score=-0.560 total time=  10.7s
[CV 3/5; 1/6] START activation_function=linear, init=uniform....................
[CV 3/5; 1/6] END activation_function=linear, init=uniform;, score=-0.590 total time=  10.6s
[CV 4/5; 1/6] START activation_function=linear, init=uniform....................
[CV 4/5; 1/6] END activation_function=linear, init=uniform;, score=-0.557 total time=  10.3s
[CV 5/5; 1/6] START activation_function=linear, init=uniform....................
[CV 5/5; 1/6] END activation_function=linear, init=uniform;, score=-0.733 total time=  10.6s
[CV 1/5; 2/6] START activation_function=linear, init=normal...........

In [17]:
#  results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.6321167290210724, using {'activation_function': 'linear', 'init': 'normal'}
-0.6492559790611268,0.10144764909463676 with: {'activation_function': 'linear', 'init': 'uniform'}
-0.6321167290210724,0.13826739430158885 with: {'activation_function': 'linear', 'init': 'normal'}
-15175.959375,820.8975043185108 with: {'activation_function': 'linear', 'init': 'zero'}
-0.7812409818172454,0.2806679310994864 with: {'activation_function': 'relu', 'init': 'uniform'}
-0.760254979133606,0.2768106584096835 with: {'activation_function': 'relu', 'init': 'normal'}
-15175.9357421875,820.9021054655276 with: {'activation_function': 'relu', 'init': 'zero'}


# Tuning : Number of neurons in activation layer

In [18]:
def create_model_neuron(neuron1,neuron2, neuron3):
  model= Sequential()
  model.add(Dense(neuron1, input_dim=10, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(neuron2, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(neuron3, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= 'normal', activation='linear')) 
  adam= Adam(learning_rate= 0.001)
# Compile model
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_neuron, verbose=0, batch_size=100, epochs=100)
neuron1=[4,8,16]
neuron2=[4,8,16]
neuron3=[4,8,16]
param_grid= dict(neuron1= neuron1, neuron2=neuron2,neuron3=neuron3 )
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv= KFold(), verbose=10 )
grid_result= grid.fit(x_standerdized,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 1/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-0.829 total time=  10.3s
[CV 2/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 2/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-0.458 total time=   9.9s
[CV 3/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 3/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-0.679 total time=  10.0s
[CV 4/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 4/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-0.555 total time=   9.8s
[CV 5/5; 1/27] START neuron1=4, neuron2=4, neuron3=4............................
[CV 5/5; 1/27] END neuron1=4, neuron2=4, neuron3=4;, score=-0.762 total time=   9.6s
[CV 1/5; 2/27] START neuron1=4, neuron2=4, neuron3=8............................
[CV 1/5; 2/27] END neuron1=

In [19]:
# summerize results
print('Best:{}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means= grid_result.cv_results_['mean_test_score']
stds= grid_result.cv_results_['std_test_score']
params= grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best:-0.5743941485881805, using {'neuron1': 8, 'neuron2': 8, 'neuron3': 4}
-0.6565497517585754,0.13508800821266556 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 4}
-0.715360414981842,0.12524571072394663 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 8}
-0.6876964569091797,0.16176416612501238 with: {'neuron1': 4, 'neuron2': 4, 'neuron3': 16}
-0.7155304729938508,0.3153554508846498 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 4}
-0.5989202797412873,0.09913509733797772 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 8}
-0.6297634661197662,0.12582272730392885 with: {'neuron1': 4, 'neuron2': 8, 'neuron3': 16}
-0.7273516058921814,0.09753856339779297 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 4}
-0.6869000434875489,0.14289946699047174 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 8}
-0.6659405946731567,0.14249319637682778 with: {'neuron1': 4, 'neuron2': 16, 'neuron3': 16}
-0.6288882791996002,0.1088695815782587 with: {'neuron1': 8, 'neuron2': 4, 'neuron3': 4}
-0.6827111601829529,0.0945671278

## Model 

In [20]:
def create_model_opt():
  model= Sequential()
  model.add(Dense(16, input_dim=10, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(8, kernel_initializer= 'normal', activation='linear'))
  model.add(Dropout(0))
  model.add(Dense(1, kernel_initializer= 'normal', activation='linear')) 
  adam= Adam(learning_rate= 0.001)
  model.compile(loss='mean_squared_error', optimizer=adam)
  return model

# Define regression model
model= KerasRegressor(build_fn=create_model_opt, verbose=10, batch_size=100, epochs=200)

estimator= model.fit(x_standerdized,y)
kfold= KFold(n_splits=10, random_state=1, shuffle=True)
results= cross_val_score(model, x_standerdized,y, cv= kfold)
print('mse:', results.mean())

Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200
Epoch 75/200
Epoch 76/200
Epoch 77/200
Epoch 78

In [21]:
print('MSE',results.mean())

MSE -0.5797735810279846
